# Before running quantmap 
## To get cid and compounds
### For table info and to start SQL

In [1]:
import sqlite3
conn = sqlite3.connect('/mnt/external-images-pvc/quantmap/qm_pr01.sqlite')
c = conn.cursor()
#c.execute("SELECT name FROM sqlite_master WHERE type='table';")
#List tables
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in c.fetchall():
    print (table)

('stitch_protchem_man',)
('string_protlink_man',)


In [2]:
c.execute("select count (distinct(taxo)) from stitch_protchem_man;")
for rows in c.fetchall():
    print (rows)
#print (len(c.fetchall()))

(1,)


In [3]:
c.execute("pragma table_info(string_protlink_man);")
#c.execute("select * from stitch_protchem_man limit 1;")
for entry in c.fetchall():
    print (entry)

(0, 'tax1', 'varchar(16)', 0, None, 0)
(1, 'pro1', 'varchar(64)', 0, None, 0)
(2, 'tax2', 'varchar(16)', 0, None, 0)
(3, 'pro2', 'varchar(64)', 0, None, 0)
(4, 'neigh', 'smallint(5)', 0, None, 0)
(5, 'fusion', 'smallint(5)', 0, None, 0)
(6, 'co_occ', 'smallint(5)', 0, None, 0)
(7, 'co_exp', 'smallint(5)', 0, None, 0)
(8, 'sc_exp', 'smallint(5)', 0, None, 0)
(9, 'sc_db', 'smallint(5)', 0, None, 0)
(10, 'sc_txt', 'smallint(5)', 0, None, 0)
(11, 'sc_all', 'smallint(5)', 0, None, 0)


In [4]:
c.execute("select * from string_protlink_man limit 5;")
for rows in c.fetchall():
    print (rows)

('9606', 'ENSP00000000233', '9606', 'ENSP00000158762', 0, 0, 0, 0, 0, 0, 619, 619)
('9606', 'ENSP00000000233', '9606', 'ENSP00000174653', 0, 0, 0, 0, 0, 0, 266, 266)
('9606', 'ENSP00000000233', '9606', 'ENSP00000203407', 0, 0, 0, 296, 0, 0, 0, 296)
('9606', 'ENSP00000000233', '9606', 'ENSP00000203630', 0, 0, 0, 249, 0, 0, 0, 249)
('9606', 'ENSP00000000233', '9606', 'ENSP00000204517', 0, 0, 0, 0, 0, 0, 224, 224)


In [12]:
c.execute("select count(cid) from stitch_protchem_man where taxo='9606';")
for rows in c.fetchall():
    print (rows)

(2626294,)


In [29]:
all_cid = []
c.execute("select distinct (cid) from stitch_protchem_man;")
for rows in c.fetchall():
    all_cid.append(rows)

In [35]:
type(all_cid[1][0])

int

### Extracting seed cid

In [39]:
%%time

seed_cid = []
for cid in all_cid:
    c.execute("select distinct protein from stitch_protchem_man where cid = " + str(cid[0]) + " and sc_all >=  700 order by sc_all desc, sc_exp desc limit 10 ")
    if len(c.fetchall()) > 0:
        seed_cid.append(cid)
    
    if len(seed_cid)%10000 == 0:
        print (len(seed_cid))

10000
10000
10000
10000
10000
10000
20000
30000
40000
50000
60000
70000
80000
90000
90000
100000
110000
110000
110000
110000
110000
110000
120000
130000
CPU times: user 27 s, sys: 46.3 s, total: 1min 13s
Wall time: 7min 45s


In [44]:
outfile = open("/scratch-shared/akshai/workdir/data_generation_quantmap/seed_cids.txt","w")
for entry in seed_cid:
    outfile.write(str(entry[0]) + "\n")
outfile.close()

### Extracting CID which has PPI

In [103]:
import time
import tqdm

In [123]:
%%time

seed_cid_file = open("/scratch-shared/akshai/workdir/data_generation_quantmap/seed_cids.txt","r").readlines()

ppi_cid = []

loop = tqdm.tqdm(enumerate(seed_cid_file),total=len(seed_cid_file),leave=False)

for i,cid in loop:
    seeds = []
    c.execute("select distinct protein from stitch_protchem_man where cid = " + str(cid[0]) + " and sc_all >=  700 order by sc_all desc, sc_exp desc limit 10 ")
    for row in c.fetchall():
        seeds.append(row[0])
        
    ppi_query = "select pro1,pro2,sc_all/1000.0 weight from string_protlink_man where pro1 in (" \
        + str(seeds)[1:-1] + ") and (pro1 < pro2 or pro2 not in (" \
        + str(seeds)[1:-1] + ")) and sc_all >= 700 order by sc_all desc, sc_exp desc, pro1, pro2  limit 150"
    
    c.execute(ppi_query)
    
    if len(c.fetchall()) > 0:
        ppi_cid.append(cid[:-1])


CPU times: user 4min 27s, sys: 43.2 s, total: 5min 10s
Wall time: 11min 28s


In [125]:
outfile = open("/scratch-shared/akshai/workdir/data_generation_quantmap/seed_cids_with_ppi.txt","w")
for entry in ppi_cid:
    outfile.write(str(entry[0]) + "\n")
outfile.close()

In [137]:
print("Number of unique CIDs = " + str(len(all_cid)))
print("Number of CIDs with seed protein = " + str(len(seed_cid_file)))
print("Number of CIDs with seed protein and have protein-protein network= " + str(len(ppi_cid)))
print("\n\n\n")

Number of unique CIDs = 279604
Number of CIDs with seed protein = 130348
Number of CIDs with seed protein and have protein-protein network= 55603






### FOR OLD DATABASE ANALYSIS

In [43]:
import time

import tqdm

import sqlite3

conn_old = sqlite3.connect('/mnt/external-images-pvc/quantmap/qm_chem_fix.sqlite') #OLD DATABASE
co = conn_old.cursor()
#c.execute("SELECT name FROM sqlite_master WHERE type='table';")
#List tables
co.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in co.fetchall():
    print (table)

('stitch_chem',)
('stitch_protchem_man',)
('string_protlink_man',)
('chem_freq',)
('chem_uniq',)
('chem_hubs_orig',)
('string_protlink_man905',)
('chem_hubs',)
('string_prots_910',)
('stitch_chem40',)
('chem_hubs_4_10',)
('akshai_chem_hubs',)
('cid_prot_rank',)
('spearman_table',)


In [45]:
co.execute("pragma table_info(akshai_chem_hubs);")
for entry in co.fetchall():
    print (entry)

co.execute("select * from akshai_chem_hubs limit 3;")
for entry in co.fetchall():
    print (entry)
    


(0, 'cid', 'INTEGER', 0, None, 0)
(1, 'V2', 'TEXT', 0, None, 0)
(2, 'dat', 'REAL', 0, None, 0)
(1, 'ENSP00000407375', 141.0, 282)
(1, 'ENSP00000355370', 140.0, 280)
(1, 'ENSP00000265643', 139.0, 278)


In [49]:
co.execute("pragma table_info(akshai_chem_hubs);")
#print (co.fetchall())
for entry in co.fetchall():
    print (entry)


co.execute("select * from akshai_chem_hubs limit 3;")
for entry in co.fetchall():
    print (entry)
    

[(0, 'cid', 'INTEGER', 0, None, 0), (1, 'V2', 'TEXT', 0, None, 0), (2, 'dat', 'REAL', 0, None, 0)]
(1, 'ENSP00000407375', 141.0, 282)
(1, 'ENSP00000355370', 140.0, 280)
(1, 'ENSP00000265643', 139.0, 278)


In [10]:

new_table =  #"chem_hubs_orig" # Give the name of new table if old table have to be changed (datatype change), else give the table for analysis

co.execute("pragma table_info(" + new_table + ");")
#c.execute("select * from stitch_protchem_man limit 1;")
for entry in co.fetchall():
    print (entry)

(0, 'cid', 'int(10)', 0, None, 0)
(1, 'protein', 'varchar(64)', 0, None, 0)
(2, 'rank', 'float', 0, None, 0)


In [4]:
# FOR NEWLY CREATED DATA OTHERWISE DO NOT EXECUTE

old_table = "akshai_chem_hubs"
#co.execute("drop table " + old_table + ";")
#co.execute("drop table " + new_table + ";")

create_table = "CREATE TABLE " + new_table + " ( cid int(10) NOT NULL,protein Varchar(100) NOT NULL ,Rank float NOT NULL );"
#co.execute(create_table)

insert_data = "INSERT INTO " + new_table + " (cid, protein, rank) SELECT cid, V2,dat FROM " + old_table + ";"
#co.execute(insert_data)

In [3]:
co.execute("select * from " + new_table + " limit 3;")
for entry in co.fetchall():
    print (entry)

(1, 'ENSP00000302665', 1.0)
(1, 'ENSP00000358525', 2.0)
(1, 'ENSP00000172229', 3.5)


In [11]:
# Get CIDs which are having calculated protein ranking data

co.execute("select cid from " + new_table + " group by cid;")
calculated_cid = []
for cid in co.fetchall():
    calculated_cid.append(cid)

In [12]:
print (str(len(calculated_cid)) + " CIDs have protein ranking data")

43538 CIDs have protein ranking data


In [13]:
# Receives two dict of compounds with their each first row as enzymeid and then the ranking
########################################
# THIS IS USING PANDA DF, BUT SLOWER   #
# THAN NORMAL IMPLEMENTATION           #
#  :(                                  #
########################################
def spearman_footrule_pd(cmpd1,cmpd2):
    df1 = pd.DataFrame.from_dict(cmpd1.items())
    df2 = pd.DataFrame.from_dict(cmpd2.items())
    df3 = pd.merge(df1,df2, on=0,how="outer")
    df4 = df3.fillna(0)
    non_zero_bool = (df4.iloc[:,1] > 0) & (df4.iloc[:,2] > 0)
    abs_sum = abs(df4[non_zero_bool].iloc[:,1] - df4[non_zero_bool].iloc[:,2])
    non_zero_count = len(abs_sum)+1
    zero_count = sum((df4 == 0).astype(int).sum(axis=1))
    summation = sum(abs_sum)
    final_value = summation + (zero_count * (non_zero_count+zero_count))
    return final_value

In [14]:
# Receives two dict of compounds with their each first row as enzymeid and then the ranking
def spearman_footrule(cmpd1,cmpd2):
    abs_diff = 0
    no_match_count_cmpd1 = 1
    match_count = 1
    for lines in cmpd1:
        eid1,rank1 = lines,cmpd1[lines]
        if eid1 in cmpd2:
            rank2 = cmpd2[eid1]
            abs_diff += abs(rank1 - rank2)
            match_count += 1
        else:
            no_match_count_cmpd1 += 1
    no_match_count_cmpd2 = (len(cmpd2)-match_count)
    no_match_count = no_match_count_cmpd1 + no_match_count_cmpd2
    return (abs_diff + (no_match_count*(match_count+no_match_count)))

In [15]:
def write_to_sql(data,table_name):
    insert_statement = "INSERT OR IGNORE INTO " + table_name + " VALUES (?,?,?);"
    co.executemany(insert_statement,data)
    conn_old.commit()

In [16]:
%%time
#GET CID,PROTEIN,RANK DATA FROM THE DB

def get_data(inp_cid):
    prot_rank_dict = {}
    co.execute("select protein,rank from " + new_table + " where cid = " + str(inp_cid) + " ; ")
    for entry in co.fetchall():
        prot_rank_dict[entry[0]] = entry[1]
    
    return (prot_rank_dict)



data_dict = {}
loop = tqdm.tqdm(enumerate(calculated_cid),total=len(calculated_cid),leave=False)
for i,cid in loop:
    cid = cid[0]
    data_dict[cid] = get_data(cid)

CPU times: user 12.3 s, sys: 10.3 s, total: 22.6 s
Wall time: 1min 40s


In [17]:
%%time

entry_to_database = True
table_name = "spearman_table"

drop_table_if_exists = False # Setting false will append unique values to the table


# Unmask if you want to drop the table if already present
def drop_existing_table():
    if entry_to_database:
        co.execute("SELECT name FROM sqlite_master WHERE type='table';")
        already_exits = False 
        for table in co.fetchall():
            if  table[0] == table_name:
                already_exits = True
                print ("Table already present")
        if already_exits:
            co.execute("drop table " + table_name + ";")
            print ("Deleted existing table")
        co.execute("Create TABLE if not exists " + table_name + "( cid1 int(10) NOT NULL, cid2 int(10) NOT NULL, spearman_number float NOT NULL,UNIQUE(cid1,cid2,spearman_number));")
        print ("New table created")

if drop_table_if_exists:
    drop_existing_table()
else:
    co.execute("Create TABLE if not exists " + table_name + "( cid1 int(10) NOT NULL, cid2 int(10) NOT NULL, spearman_number float NOT NULL,UNIQUE(cid1,cid2,spearman_number));")


cid_list = calculated_cid[4597+2035+2052+2388+1710+12797+2270+3060:]

finished_dict = {}
query_time = 0
loop = tqdm.tqdm(enumerate(cid_list),total=len(cid_list),leave=False)

output_list = []
for i,cid1 in loop:
    #print (len(cid_list[i:]))
    cid1 = cid1[0]
    cmpd1 = data_dict[cid1]
    
    if not entry_to_database:
        finished_dict[cid1] = {}
        
    for j,cid2 in enumerate(cid_list[i:]):
        cid2 = cid2[0]
        if cid1 != cid2:# and cid2 not in done_loop1:
            cmpd2 = data_dict[cid2]
            start = time.time()
            
            if entry_to_database:
                spearman_value = spearman_footrule(cmpd1,cmpd2)
                output_list.append((cid1,cid2,spearman_value))
            else:
                finished_dict[cid1][cid2] = (spearman_footrule(cmpd1,cmpd2))
            
            diff = time.time() - start
            query_time += diff
    
    if entry_to_database and (i+1) % 10 == 0:
        write_to_sql(output_list,table_name)
        output_list = []
        
    if (i + 1) % 100 == 0:
        checkpoint = open("loop_checkpoint.txt","w")
        checkpoint.write(str(i) + "\n")
        checkpoint.close()
    
print ("Spearman footrule calculation time = " + str(query_time) + " s")

KeyboardInterrupt: 

In [3]:
co.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in co.fetchall():
    print (table)

('stitch_chem',)
('stitch_protchem_man',)
('string_protlink_man',)
('chem_freq',)
('chem_uniq',)
('chem_hubs_orig',)
('string_protlink_man905',)
('chem_hubs',)
('string_prots_910',)
('stitch_chem40',)
('chem_hubs_4_10',)
('akshai_chem_hubs',)
('cid_prot_rank',)
('spearman_table',)


In [4]:
co.execute("pragma table_info(spearman_table);")
for table in co.fetchall():
    print (table)

(0, 'cid1', 'int(10)', 1, None, 0)
(1, 'cid2', 'int(10)', 1, None, 0)
(2, 'spearman_number', 'float', 1, None, 0)


#### Checking the validity of the data and missing data

In [7]:
co.execute("select count(*) from spearman_table ;")
#for entry in co.fetchall():
#    print (entry)
print (co.fetchall())

[(947756212,)]


In [12]:
group1 = {}
co.execute("select cid1,count(cid2) from spearman_table group by cid1;")
for entry in co.fetchall():
    group1[entry[0]] = entry[1]
#print (co.fetchall())


In [13]:
group2 = {}
co.execute("select cid2,count(cid1) from spearman_table group by cid2;")
for entry in co.fetchall():
    group2[entry[0]] = entry[1]
#print (co.fetchall())


In [14]:
print (len(group1),len(group2))

43499 43537


In [21]:
not_in_group2 = []
summation_dict = {}
for i,entry in enumerate(group1):
    if entry in group2:
        summation = group1[entry] + group2[entry]
        summation_dict[entry] = summation
    else:
        not_in_group2.append(entry)

In [30]:
for entry in summation_dict:
    if summation_dict[entry] != 43537:
        print (entry)
        break

In [31]:
### Finding the min,max value from data
co.execute("select max(spearman_number),min(spearman_number) from spearman_table;")

In [32]:
print (co.fetchall())
minimum_value = 0
maximum_value = 102080

[(102080.0, 0.0)]


In [57]:
co.execute("ALTER TABLE spearman_table ADD COLUMN relative_distance GENERATED ALWAYS AS (spearman_number/102080.0);")
conn_old.commit()
co.execute("SELECT sql FROM sqlite_master WHERE tbl_name = 'spearman_table' AND type = 'table';")
print (co.fetchall())
    
co.execute("select * from spearman_table limit 3;")
for entry in co.fetchall():
    print (entry)

[('CREATE TABLE spearman_table( cid1 int(10) NOT NULL, cid2 int(10) NOT NULL, spearman_number float NOT NULL, relative_distance GENERATED ALWAYS AS (spearman_number/102080.0),UNIQUE(cid1,cid2,spearman_number))',)]
(1, 6, 46101.5, 0.45162127742946706)
(1, 7, 45551.5, 0.44623334639498435)
(1, 8, 25658.5, 0.2513567789968652)


In [216]:
co.execute("select count(*) from spearman_table where relative_distance = 0;")
print (co.fetchall())

[(1794252,)]


In [148]:
# baum ragan matrix

### For extracting all the compounds in the database

In [48]:
c.execute("select cid from stitch_protchem_man group by cid;")
query = c.fetchall()
output_filename = str(len(query)) + "_compounds.txt"
output = open(output_filename,"w")
for entry in query:
    output.write(str(entry[0]) + "\tCID" + str(entry[0]) + "\n")
output.close()

### For extracting set number of compounds

In [53]:
number_of_compounds = 1000

c.execute("select cid from stitch_protchem_man group by cid limit "\
          + str(number_of_compounds) + ";")
query = c.fetchall()

output_filename = str(len(query)) + "_compounds.txt"
output = open(output_filename,"w")
for entry in query:
    output.write(str(entry[0]) + "\tCID" + str(entry[0]) + "\n")
output.close()

### Convert input names to CID

## Run quantmap

In [54]:
import os
os.system("Rscript requant.r " + output_filename + \
          " " + output_filename[:-4] + ".png")

512

# After running quantmap

In [60]:
open_file = open("newick_format.txt","r").readlines()
newick_format = ""
for entry in open_file:
    newick_format += entry[:-1]

In [61]:
total_branches = newick_format.count("(") - 1
print ("Total number of branches = " + str(total_branches))

Total number of branches = 1489


In [62]:
newick_format = newick_format.replace("CID","")
write_newick = open("newick_format_output.txt","w")
write_newick.write(newick_format)
write_newick.close()

In [1]:
import pandas as pd
distance_matrix = "distance_matrix_5338.txt"
output_matrix = distance_matrix[:-4] + "_out.csv"
open_file = open(distance_matrix,"r").readlines()
distance_matrix = {}
output = ""
for count,line in enumerate(open_file):
    if count == 0:
        heading_list = line.split(",")
        output += "," + line
    else:
        output += heading_list[count-1].strip() + "," + line
    
output = output.replace("CID","")
out_file = open(output_matrix,"w")
out_file.write(output)
out_file.close()


In [6]:
m = pd.read_csv(output_matrix,index_col=0)
m

,1,6,7,8,9,10,11,13,14,15,...,17429,17472,17473,17507,17513,17516,17520,17532,17534,17596
1,0.000000,0.454521,0.449010,0.252997,0.560453,0.535567,0.613794,0.574930,0.263520,0.629182,...,0.579696,0.842437,0.453560,0.699010,0.523403,0.640218,0.252292,0.560162,0.843398,0.398745
6,0.454521,0.000000,0.236968,0.099564,0.390213,0.296741,0.493568,0.204886,0.100905,0.423774,...,0.491946,0.547446,0.177254,0.506679,0.416741,0.411314,0.111167,0.348638,0.357840,0.193441
7,0.449010,0.236968,0.000000,0.091796,0.396286,0.283216,0.488481,0.301951,0.093088,0.431306,...,0.427811,0.542083,0.220323,0.489008,0.389770,0.476558,0.105597,0.413660,0.498147,0.182493
8,0.252997,0.099564,0.091796,0.000000,0.202416,0.120941,0.278267,0.143136,0.017803,0.235307,...,0.276572,0.314091,0.086177,0.283216,0.217385,0.278272,0.022239,0.231285,0.289939,0.065476
9,0.560453,0.390213,0.396286,0.202416,0.000000,0.454693,0.573116,0.496836,0.211569,0.608180,...,0.644462,0.778844,0.384515,0.633150,0.579696,0.635156,0.202953,0.548506,0.748211,0.328547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17516,0.640218,0.411314,0.476558,0.278272,0.635156,0.572840,0.710667,0.606987,0.281072,0.677170,...,0.793996,0.915158,0.488023,0.811888,0.714600,0.000000,0.288796,0.586365,0.890164,0.430818
17520,0.252292,0.111167,0.105597,0.022239,0.202953,0.143136,0.278277,0.160249,0.024152,0.247866,...,0.269445,0.334526,0.099564,0.293493,0.221905,0.288796,0.000000,0.235376,0.314091,0.077207
17532,0.560162,0.348638,0.413660,0.231285,0.548506,0.503780,0.619093,0.435544,0.233025,0.580134,...,0.666238,0.764733,0.333230,0.633875,0.607056,0.586365,0.235376,0.000000,0.658352,0.365933
17534,0.843398,0.357840,0.498147,0.289939,0.748211,0.414370,0.864193,0.289845,0.291807,0.756881,...,0.869551,0.935239,0.307476,0.890164,0.662561,0.890164,0.314091,0.658352,0.000000,0.433637


In [7]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering as AC
import numpy as np
kmean = ""
kmean=AC(n_clusters=500, affinity='precomputed',linkage="complete")
kmean.fit(m)
result= kmean.labels_
m['label']= result
cid_cluster3 = m['label']

In [8]:
def find_cluster_overlap(dict1,dict2):
    
    dict_1_clusters = {dict1[entry] : [] for entry in dict1.keys()}
    for entry in dict1.keys():
        dict_1_clusters[dict1[entry]].append(entry)
    
    dict_2_clusters = {dict2[entry] : [] for entry in dict2.keys()}
    for entry in dict2.keys():
        dict_2_clusters[dict2[entry]].append(entry)
    
    all_label_clustered = {dict1[entry] : [0 for entry in dict_2_clusters.keys()] for entry in dict1.keys()}
    for label1 in dict_1_clusters:
        for label2 in dict_2_clusters:
            for entry in dict_1_clusters[label1]:
                if entry in dict_2_clusters[label2]:
                    all_label_clustered[label1][label2] += 1
    
    total = 0
    for entry in all_label_clustered:
        total += sum(all_label_clustered[entry])
    print (total,all_label_clustered)
find_cluster_overlap(cid_cluster2,cid_cluster1)


NameError: name 'cid_cluster2' is not defined

In [137]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering as AC
import numpy as np
kmean = ""
kmean=AC(n_clusters=5, affinity='precomputed',linkage="complete")
kmean.fit(m)
result= kmean.labels_
m['label']=result
cid_cluster1 = m['label']

In [217]:
all_cids = [ entry for entry in cid_cluster1.keys()]

### Trial clustering

In [245]:
all_cids = []
for cid in m:
    all_cids.append(cid)

In [246]:
table_to_analyse = "chem_hubs_orig"

co.execute("pragma table_info(" + table_to_analyse + ");")
for entry in co.fetchall():
    print (entry)

co.execute("select * from " + table_to_analyse + " limit 3;")
query_output = co.fetchall()
for entry in query_output:
    print (entry)

co.execute("select cid,protein,rank from " + table_to_analyse + ";")
sql_data = co.fetchall()
print (len(sql_data))

(0, 'cid', 'int(10)', 0, None, 0)
(1, 'protein', 'varchar(64)', 0, None, 0)
(2, 'rank', 'float', 0, None, 0)
(1, 'ENSP00000302665', 1.0)
(1, 'ENSP00000358525', 2.0)
(1, 'ENSP00000172229', 3.5)
4227856


In [247]:
unique_protein_ids = []
for entry in sql_data:
    unique_protein_ids.append(entry[1])

In [248]:
wrd_indx = {}
indx_wrd = {}
for i,entry in enumerate(set(unique_protein_ids)):
    wrd_indx[entry] = i
    indx_wrd[i] = entry

In [249]:
def make_full_feature(data,cid):
    output_list = [0 for entry in wrd_indx]

    for entry in data:
        if entry[0] == cid:
            output_list[wrd_indx[entry[1]]] = entry[2]
    
    return (output_list)


In [261]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np

batch_size = 1000
cid_list = all_cids #list(set([entry[0] for entry in sql_data]))
batches = len(cid_list)//batch_size

kmeans = MiniBatchKMeans(n_clusters=5,random_state=0,batch_size=batch_size)

In [262]:
loop = tqdm.tqdm(range(batches),total=batches,leave=False)
for batch in loop:
    batch_data = []
    for count in range(batch_size):
        batch_data.append(make_full_feature(sql_data,cid_list[count+((batch)*batch_size)]))
    kmeans = kmeans.partial_fit(np.array(batch_data))

In [272]:
x_y = {}
loop = tqdm.tqdm(cid_list,total=len(cid_list),leave=False)
for cid in loop:
    x_y[cid] = kmeans.predict(np.array([make_full_feature(sql_data,cid)]))[0]

KeyboardInterrupt: 

In [308]:
x_y_new = {}
for entry in x_y:
    x_y_new[int(entry)] = int(x_y[entry])

In [309]:
for key,label in zip(cid_cluster1.keys(),cid_cluster1):
    if key in x_y_new:
        print (x_y_new[key],label)

0 0
0 2
0 1
0 2
0 0
0 2
0 4
0 2
0 2
0 4
0 3
0 2
0 2
0 2
0 2
0 2
0 1
0 2
0 2
0 0
0 2
0 2
0 2
0 2
0 2
0 2
0 2
0 1
0 2
0 2
0 2
0 2
0 4
0 4
0 2
0 0
0 1
0 2
0 0
0 2
0 2
0 2
0 2
0 2
0 0
0 2
0 2
0 1
0 2
0 2
0 1
0 2
0 2
0 2
0 2
0 0
0 2
0 2
0 2
0 1
0 1
0 0
0 2
0 1
0 2
0 3
0 2
0 2
0 2
0 2
0 1
0 2
0 2
0 2
0 2
0 1
0 2
0 2
0 2
0 2
0 2
0 2
0 2
0 1
0 2
0 1
0 1
0 2
0 1
0 2
0 2
0 1
0 1
0 1
0 1
0 2
0 2
0 2
0 4
0 2
0 1
0 2
0 2
0 2
0 2
0 0
0 1
0 1
0 2
0 2
0 2
0 2
0 2
0 2
0 0
0 1
0 2
0 1
0 1
0 0
0 2
0 1
0 1
0 4
0 0
0 0
0 1
0 2
0 2
0 2
0 1
0 1
0 2
0 1
0 2
0 2
0 1
0 0
0 2
0 2
0 2
0 2
0 2
0 1
0 2
0 0
0 2
0 1
0 2
0 1
0 1
0 0
0 1
0 0
0 0
0 1
0 2
0 1
0 1
0 0
0 1
0 2
0 2
0 2
0 2
0 2
0 1
0 1
0 0
0 1
0 0
0 2
0 2
0 1
0 2
0 0
0 4
0 2
0 0
0 2
0 0
0 1
0 2
0 2
0 2
0 2
0 2
0 1
0 1
0 0
0 4
0 1
0 1
0 1
0 1
0 2
0 1
0 0
0 4
0 1
0 2
0 2
0 1
0 0
0 2
0 2
0 2
0 1
0 1
0 2
0 2
0 1
0 2
0 2
0 2
0 2
0 2
0 2
0 4
0 2
0 2
0 1
0 2
0 1
0 2
0 2
0 2
0 2
0 1
0 1
0 2
0 0
0 2
0 1
0 2
0 2
0 1
0 0
0 0
0 2
0 2
0 2
0 2
0 2
0 2
0 2
0 0
0 1
0 1
0 2


In [306]:
x_y['        1']

0

0

In [9]:
#table info
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('stitch_protchem_man',), ('string_protlink_man',)]


In [10]:
index = cid_cluster.index
cid_list = list(index)
cid_list = list(map(int, cid_list))

In [11]:
connect = sqlite3.connect('/mnt/external-images-pvc/quantmap/qm_chem_fix.sqlite')
d = connect.cursor()
d.execute("select cid,smiles from stitch_chem where cid in ("+str(cid_list)[1:-1]+");")
cid_smiles_list = d.fetchall()

In [12]:
cid_smiles_dict = {}
for entry in cid_smiles_list:
    cid_smiles_dict[entry[0]] = entry[1]

In [13]:
output = open("ML_input_5338.txt","w")
cluster_count = {}
for cid,cluster in zip(cid_list,cid_cluster):
    try:
        output.write(str(cid_smiles_dict[cid]) + "\t" + str(cluster) + "\n")
        if cluster not in cluster_count:
            cluster_count[cluster] = 1
        else:
            cluster_count[cluster] += 1
    except:
        pass
output.close()

In [14]:
cluster_count

{0: 756, 2: 1971, 1: 1970, 4: 258, 3: 268}